In [1]:

import cv2 as cv
import numpy as np
from keras.models import model_from_json
import tensorflow as tf
import pyautogui as pgi

In [2]:
cam=cv.VideoCapture(0)
minValue=70
interrupt=-1
class_names=['Forward', 'Next', 'Pause', 'Play', 'Previous', 'Reverse', 'Volume Down', 'Volume Up']
if not cam.isOpened():
    print("Error!Camera is not opened.")
    exit()

In [3]:
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("model-bw.h5")


In [4]:
s=2
x_prev="Play"
c=0
while True:
    ret,frame = cam.read()
    frame = cv.flip(frame, 1)
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    cv.rectangle(frame, (220-1, 9), (720+1, 519), (255,0,0) ,1)
    roi = frame[10:510, 220:720]
    
    gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(5,5),2)
    th3 = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY_INV,11,2)
    ret, test_image = cv.threshold(th3, minValue, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    test_image = cv.resize(test_image, (128,128))
    
    img_array = tf.keras.utils.img_to_array(test_image)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    x=class_names[np.argmax(score)]
    if(x==x_prev):
        c=c+1
    if(c>=20):
        cv.putText(frame,x,(10,500), cv.FONT_HERSHEY_PLAIN, 1.25, (55,0,255), 2)
        if(x=="Play" and s==2):
            pgi.press('space')
            s=1
        elif(x=="Pause" and s==1):
            pgi.press('space')
            s=2
        elif(x=="Volume Up"):
            pgi.press('up')
        elif(x=="Volume Down"):
            pgi.press('down')
        elif(x=="Forward"):
            pgi.press("right")
        elif(x=="Reverse"):
            pgi.press('left')
        elif(x=="Next"):
            pgi.hotkey('command','right')
        elif(x=="Previous"):
            pgi.hotkey('command','left')
        c=0       
    x_prev=x
        
        
            
    cv.imshow("Frame", frame)
    cv.imshow("roi", test_image)
    interrupt = cv.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    
cam.release()
cv.destroyAllWindows()